In [8]:
import os
import pandas as pd
import numpy as np
import gradio as gr
from joblib import load

In [9]:

BEST_MODEL_PATH = os.path.join('models', 'best_model.joblib')
ENCODERS_PATH = os.path.join('models', 'label_encoders.joblib')

try:
    best_model = load(BEST_MODEL_PATH)
    print(f'Loaded best model from {BEST_MODEL_PATH}')
except Exception as e:
    best_model = None
    print(f'Error loading best model ({BEST_MODEL_PATH}): {e}')

try:
    encoders = load(ENCODERS_PATH)
    print(f'Loaded encoders from {ENCODERS_PATH}')
except Exception as e:
    encoders = None
    print(f'Error loading encoders ({ENCODERS_PATH}): {e}')

DEFAULT_FEATURES = [
    'Specific_Gravity',
    'Albumin',
    'Hemoglobin',
    'Packed_Cell_Volume',
    'Red_Blood_Cell_Count',
    'Hypertension',
    'Diabetes_Mellitus'
]

Loaded best model from models\best_model.joblib
Loaded encoders from models\label_encoders.joblib


In [10]:
def _build_input_df(values, feature_names=None):
    """Builds a single-row DataFrame from values. If feature_names is given, reorder to match."""
    df = pd.DataFrame([values], columns=DEFAULT_FEATURES)

    if feature_names is not None:
        try:
            req = list(feature_names)
            for c in req:
                if c not in df.columns:
                    df[c] = np.nan
            df = df[req]
        except Exception:
            pass

    return df


def predict_api(Specific_Gravity, Albumin, Hemoglobin, Packed_Cell_Volume, Red_Blood_Cell_Count, Hypertension, Diabetes_Mellitus):
    """Predict function used by Gradio. Returns a human-readable string."""
    values = [Specific_Gravity, Albumin, Hemoglobin, Packed_Cell_Volume, Red_Blood_Cell_Count, Hypertension, Diabetes_Mellitus]

    if best_model is None:
        return 'No saved model found at models/best_model.joblib. Please train and save the model first.'

    df = _build_input_df(values)

    if encoders is None:
        for col in ['Hypertension', 'Diabetes_Mellitus']:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: 1 if str(x).lower() == 'yes' else 0)
    else:
        for col, le in encoders.items():
            if col in df.columns:
                df[col] = df[col].fillna('missing')
                try:
                    df[col] = le.transform(df[col].astype(str))
                except Exception:
                    df[col] = df[col].astype(str).apply(lambda x: x if x in le.classes_ else 'missing')
                    df[col] = le.transform(df[col])

    model_feature_names = getattr(best_model, 'feature_names_in_', None)
    if model_feature_names is not None:
        req = list(model_feature_names)
        for c in req:
            if c not in df.columns:
                df[c] = np.nan
        df = df[req]
    else:
        df = df[DEFAULT_FEATURES]

    try:
        pred = best_model.predict(df)
    except Exception as e:
        return f'Error during prediction with model: {e}'

    try:
        label = int(pred[0])
    except Exception:
        return f'Unexpected prediction output: {pred}'

    if label == 0:
        return 'You have Chronic kidney disease (CKD)'
    else:
        return "You don't have Chronic kidney disease (CKD)"


inputs = [
    gr.Number(label='Specific Gravity', value=1.02),
    gr.Number(label='Albumin', value=0),
    gr.Number(label='Hemoglobin', value=15.0),
    gr.Number(label='Packed Cell Volume', value=40),
    gr.Number(label='Red Blood Cell Count', value=5.0),
    gr.Radio(choices=['yes', 'no'], label='Hypertension', value='no'),
    gr.Radio(choices=['yes', 'no'], label='Diabetes Mellitus', value='no')
]
output = gr.Textbox(label='Kidney Disease Prediction')

gr.Interface(fn=predict_api, inputs=inputs, outputs=output, title='Kidney Disease Prediction').launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.

To create a public link, set `share=True` in `launch()`.
